In [3]:
from utils import Graph, Node, Edge, GraphBuilder

BASE_PATH = '../data/world'
NODES_PATH = BASE_PATH + '/world_nodes.txt'
EDGES_PATH = BASE_PATH + '/world_edges.txt'

GraphBuilder = GraphBuilder(NODES_PATH, EDGES_PATH)
graph = GraphBuilder.build_graph()


In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import AgglomerativeClustering

def assign_warehouses(graph, central_criteria, regional_criteria, local_criteria):
    nodes = graph.nodes
    edges = graph.edges

    num_clusters = [central_criteria, regional_criteria, local_criteria]
    assigned_clusters = []

    nodes_coordinates = [(node.x, node.y) for node in nodes]
    nodes_coordinates = np.array(nodes_coordinates)

    for n_clusters in num_clusters:
        cluster_model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
        clusters = cluster_model.fit_predict(nodes_coordinates)
        assigned_clusters.append(clusters)

    distances = np.full((len(nodes), len(nodes)), fill_value=np.inf)

    for edge in edges:
        from_node = edge.node1
        to_node = edge.node2
        weight_info = edge.weight

        weight = 0
        for weight_type, weight in weight_info.items():\
            weight+=weight

           

        from_idx = nodes.index(from_node)
        to_idx = nodes.index(to_node)


        distances[from_idx][to_idx] = weight
        distances[to_idx][from_idx] = weight

    num_central = len(set(assigned_clusters[0]))
    num_regional = len(set(assigned_clusters[1]))
    num_local = len(set(assigned_clusters[2]))

    cost_matrix = np.full((num_central + num_regional + num_local, len(distances)), fill_value=np.inf)

    for i, node in enumerate(nodes_coordinates):
        for j in range(num_central):
            if assigned_clusters[0][i] == j:
                cost_matrix[j, i] = sum(distances[i, k] for k in range(len(nodes)))
            for k in range(num_regional):
                if assigned_clusters[1][i] == k:
                    cost_matrix[num_central + k, i] = sum(distances[i, k] for k in range(len(nodes)))
                for l in range(num_local):
                    if assigned_clusters[2][i] == l:
                        cost_matrix[num_central + num_regional + l, i] = sum(distances[i, k] for k in range(len(nodes)))

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    assignments = {
        'central': [],
        'regional': [],
        'local': []
    }

    for i, node in enumerate(nodes_coordinates):
        assignment_type = None
        assignment_idx = None

        if row_ind[i] < num_central:
            assignment_type = 'central'
            assignment_idx = row_ind[i]
        elif row_ind[i] < num_central + num_regional:
            assignment_type = 'regional'
            assignment_idx = row_ind[i] - num_central
        else:
            assignment_type = 'local'
            assignment_idx = row_ind[i] - num_central - num_regional

        assignments[assignment_type].append((nodes[i], assignment_idx))

    return assignments

central_criteria = 5
regional_criteria = 50
local_criteria = 500

assignments = assign_warehouses(graph, central_criteria, regional_criteria, local_criteria)

central_warehouse_assignments = assignments['central']

regional_warehouse_assignments = assignments['regional']
local_warehouse_assignments = assignments['local']